In [1]:
import os 
import sys 
parent_dir = os.path.dirname(os.getcwd())
# sys.path.append("/home/jbc0071/Documents/CANIDS/code")
sys.path.append(parent_dir + "/code")

from model.CANnoloAttackDetector import CANnoloAttackDetector
from data_helpers.CANDataset import CANDataset
from helpers import calculate_feature_vec_length, seperate_attack_loader, calculate_metrics
from dotenv import load_dotenv
# from data_helpers.CANDataLoader import CANDataLoader
import os

load_dotenv()
data_path = os.getenv('DATA_PATH')
dataset = CANDataset(data_path, log_verbosity=1)

config = {
    "batch_size": 64,
    # "delta_time_last_msg": {
    #     "specific_to_can_id": False,
    #     "records_back": 30
    # },
    "delta_time_last_same_aid": {
        "specific_to_can_id": True,
        "records_back": 10
    },
}

ambient_loader, validation_loader, attack_loader = dataset.get_dataloaders(config)

attack_loaders = seperate_attack_loader(attack_loader, config, remove_non_labelled=True)


Found ambient and attack directories.
Loading CAN metadata...
Parquet files found...
Found processed parquet files...
Loading processed parquet files...
Loading processing data into 'CANData' structure
No attack labels in accelerator_attack_drive_1.log
No attack labels in accelerator_attack_drive_2.log
No attack labels in accelerator_attack_reverse_1.log
No attack labels in accelerator_attack_reverse_2.log
Found attack labels in correlated_signal_attack_1.log
Found attack labels in correlated_signal_attack_2.log
Found attack labels in correlated_signal_attack_3.log
Found attack labels in fuzzing_attack_1.log
Found attack labels in fuzzing_attack_2.log
Found attack labels in fuzzing_attack_3.log
Found attack labels in max_engine_coolant_temp_attack.log
Found attack labels in max_speedometer_attack_1.log
Found attack labels in max_speedometer_attack_2.log
Found attack labels in max_speedometer_attack_3.log
Found attack labels in reverse_light_off_attack_1.log
Found attack labels in rever

In [2]:
display(attack_loaders)

In [4]:

unique_can_ids = dataset.get_unique_can_ids()
num_can_ids = len(unique_can_ids)
feature_vec_length = ambient_loader.features_len

# Load model
# model_path = 'models/canolo_model_112.pt'
model_path = 'models/64.pt'

model_config = {
    "embedding_dim": num_can_ids,
    "lstm_units": 128,
    "dense_units": 256,
    "dropout_rate": 0.2,
    "num_embeddings": max(unique_can_ids) + 1, # not sure why + 1 rn but it works
    "feature_vec_length": calculate_feature_vec_length(config)
}


In [5]:
start = 0.0001
end = 0.0000001
num_steps = 60  # or however many steps you want

# Calculate step size
step = (end - start) / num_steps

# Generate thresholds
thresholds = [start + step * i for i in range(num_steps + 1)]

print(thresholds)

[0.0001, 9.8335e-05, 9.667000000000001e-05, 9.5005e-05, 9.334000000000001e-05, 9.1675e-05, 9.001e-05, 8.834500000000001e-05, 8.668e-05, 8.5015e-05, 8.335e-05, 8.1685e-05, 8.002000000000001e-05, 7.8355e-05, 7.669e-05, 7.502500000000001e-05, 7.336e-05, 7.169500000000001e-05, 7.003e-05, 6.8365e-05, 6.670000000000001e-05, 6.5035e-05, 6.337000000000001e-05, 6.1705e-05, 6.004000000000001e-05, 5.837500000000001e-05, 5.6710000000000004e-05, 5.5045000000000006e-05, 5.338000000000001e-05, 5.171500000000001e-05, 5.0050000000000004e-05, 4.8385000000000005e-05, 4.672000000000001e-05, 4.505500000000001e-05, 4.339000000000001e-05, 4.1725000000000005e-05, 4.0060000000000006e-05, 3.8395e-05, 3.673000000000001e-05, 3.5065000000000004e-05, 3.340000000000001e-05, 3.173500000000001e-05, 3.0070000000000002e-05, 2.840500000000001e-05, 2.6740000000000005e-05, 2.5075000000000013e-05, 2.3410000000000008e-05, 2.1745000000000003e-05, 2.008000000000001e-05, 1.8415000000000006e-05, 1.6750000000000014e-05, 1.5085000

In [6]:
from concurrent.futures import ProcessPoolExecutor
from detect import detect_wrapper
import time

num_attack_files = len(attack_loaders)
detectors = []

start_time = time.time()
for _ in range(num_attack_files):
    detector = CANnoloAttackDetector(model_path, thresholds, model_config, force_cpu=True)
    detectors.append(detector)

with ProcessPoolExecutor() as executor:
    list_of_filename_and_results = list(executor.map(detect_wrapper, zip(attack_loaders, detectors)))

for loader in attack_loaders:
    loader.reset()
end_time = time.time()

print(f"Finished in {end_time - start_time} seconds")

print(list_of_filename_and_results)




Forcing CPU usage...
Forcing CPU usage...
Forcing CPU usage...
Forcing CPU usage...
Forcing CPU usage...
Forcing CPU usage...
Forcing CPU usage...
Forcing CPU usage...
Forcing CPU usage...
Forcing CPU usage...
Forcing CPU usage...
Forcing CPU usage...
Forcing CPU usage...
Forcing CPU usage...
Forcing CPU usage...
Forcing CPU usage...
Testing on correlated_signal_attack_1.log
Testing on reverse_light_off_attack_1.log
Testing on reverse_light_off_attack_2.log
Testing on correlated_signal_attack_2.log
Testing on fuzzing_attack_3.log
Testing on max_speedometer_attack_1.log
Testing on fuzzing_attack_1.log
Testing on correlated_signal_attack_3.log
Testing on max_engine_coolant_temp_attack.log
Testing on reverse_light_off_attack_3.log
Testing on reverse_light_on_attack_1.log
Testing on fuzzing_attack_2.log
Testing on max_speedometer_attack_3.log
Testing on max_speedometer_attack_2.log
Testing on reverse_light_on_attack_2.log
Testing on reverse_light_on_attack_3.log


TypeError: '>' not supported between instances of 'Tensor' and 'list'

In [5]:
display(list_of_filename_and_results)

[('correlated_signal_attack_1.log',
  {9.5e-05: (0.9597595548089038, 0.0, 0.0, 0, [[0, 980], [2086, 73126]]),
   8.5e-05: (0.9597595548089038, 0.0, 0.0, 0, [[0, 980], [2086, 73126]]),
   7.5e-05: (0.9597595548089038, 0.0, 0.0, 0, [[0, 980], [2086, 73126]]),
   6.5e-05: (0.9597595548089038, 0.0, 0.0, 0, [[0, 980], [2086, 73126]]),
   5.5e-05: (0.9597595548089038, 0.0, 0.0, 0, [[0, 980], [2086, 73126]])}),
 ('correlated_signal_attack_2.log',
  {9.5e-05: (0.9562928501469148, 0.0, 0.0, 0, [[0, 717], [2139, 62488]]),
   8.5e-05: (0.9562928501469148, 0.0, 0.0, 0, [[0, 717], [2139, 62488]]),
   7.5e-05: (0.9562928501469148, 0.0, 0.0, 0, [[0, 717], [2139, 62488]]),
   6.5e-05: (0.9562622428991185, 0.0, 0.0, 0, [[0, 719], [2139, 62486]]),
   5.5e-05: (0.9562469392752204, 0.0, 0.0, 0, [[0, 720], [2139, 62485]])}),
 ('correlated_signal_attack_3.log',
  {9.5e-05: (0.9578316326530613, 0.0, 0.0, 0, [[0, 390], [1263, 37547]]),
   8.5e-05: (0.9578061224489796, 0.0, 0.0, 0, [[0, 391], [1263, 37546]]),


In [10]:
# # structure:
# (filename, (accuracy, precision, recall, f1_score, confusion_matrix)
# ex:
# ('reverse_light_on_attack_3.log', (0.972242727471566, 0.0, 0.0, 0, [[0, 1710], [2351, 142243]]))

# graph thresholds vs avg f1 score for that group

# for the best performing threshold make a graph for each filename and show the results

list_of_results = [('correlated_signal_attack_1.log', {9.5e-05: (0.9597595548089038, 0.0, 0.0, 0, [[0, 980], [2086, 73126]]), 8.5e-05: (0.9597595548089038, 0.0, 0.0, 0, [[0, 980], [2086, 73126]]), 7.5e-05: (0.9597595548089038, 0.0, 0.0, 0, [[0, 980], [2086, 73126]]), 6.5e-05: (0.9597595548089038, 0.0, 0.0, 0, [[0, 980], [2086, 73126]]), 5.5e-05: (0.9597595548089038, 0.0, 0.0, 0, [[0, 980], [2086, 73126]])}), ('correlated_signal_attack_2.log', {9.5e-05: (0.9562928501469148, 0.0, 0.0, 0, [[0, 717], [2139, 62488]]), 8.5e-05: (0.9562928501469148, 0.0, 0.0, 0, [[0, 717], [2139, 62488]]), 7.5e-05: (0.9562928501469148, 0.0, 0.0, 0, [[0, 717], [2139, 62488]]), 6.5e-05: (0.9562622428991185, 0.0, 0.0, 0, [[0, 719], [2139, 62486]]), 5.5e-05: (0.9562469392752204, 0.0, 0.0, 0, [[0, 720], [2139, 62485]])}), ('correlated_signal_attack_3.log', {9.5e-05: (0.9578316326530613, 0.0, 0.0, 0, [[0, 390], [1263, 37547]]), 8.5e-05: (0.9578061224489796, 0.0, 0.0, 0, [[0, 391], [1263, 37546]]), 7.5e-05: (0.9578061224489796, 0.0, 0.0, 0, [[0, 391], [1263, 37546]]), 6.5e-05: (0.9578061224489796, 0.0, 0.0, 0, [[0, 391], [1263, 37546]]), 5.5e-05: (0.9578061224489796, 0.0, 0.0, 0, [[0, 391], [1263, 37546]])}), ('fuzzing_attack_1.log', {9.5e-05: (0.9910745614035088, 0.002680965147453083, 0.027777777777777776, 0.004889975550122249, [[1, 372], [35, 45192]]), 8.5e-05: (0.9910745614035088, 0.002680965147453083, 0.027777777777777776, 0.004889975550122249, [[1, 372], [35, 45192]]), 7.5e-05: (0.9910745614035088, 0.002680965147453083, 0.027777777777777776, 0.004889975550122249, [[1, 372], [35, 45192]]), 6.5e-05: (0.9910745614035088, 0.002680965147453083, 0.027777777777777776, 0.004889975550122249, [[1, 372], [35, 45192]]), 5.5e-05: (0.9910745614035088, 0.002680965147453083, 0.027777777777777776, 0.004889975550122249, [[1, 372], [35, 45192]])}), ('fuzzing_attack_2.log', {9.5e-05: (0.97625, 0.004322766570605188, 0.2, 0.00846262341325811, [[3, 691], [12, 28894]]), 8.5e-05: (0.97625, 0.004322766570605188, 0.2, 0.00846262341325811, [[3, 691], [12, 28894]]), 7.5e-05: (0.97625, 0.004322766570605188, 0.2, 0.00846262341325811, [[3, 691], [12, 28894]]), 6.5e-05: (0.9761824324324324, 0.004310344827586207, 0.2, 0.008438818565400843, [[3, 693], [12, 28892]]), 5.5e-05: (0.9761824324324324, 0.004310344827586207, 0.2, 0.008438818565400843, [[3, 693], [12, 28892]])}), ('fuzzing_attack_3.log', {9.5e-05: (0.9901666666666666, 0.008547008547008548, 0.3333333333333333, 0.016666666666666666, [[1, 116], [2, 11881]]), 8.5e-05: (0.9901666666666666, 0.008547008547008548, 0.3333333333333333, 0.016666666666666666, [[1, 116], [2, 11881]]), 7.5e-05: (0.9901666666666666, 0.008547008547008548, 0.3333333333333333, 0.016666666666666666, [[1, 116], [2, 11881]]), 6.5e-05: (0.9901666666666666, 0.008547008547008548, 0.3333333333333333, 0.016666666666666666, [[1, 116], [2, 11881]]), 5.5e-05: (0.9901666666666666, 0.008547008547008548, 0.3333333333333333, 0.016666666666666666, [[1, 116], [2, 11881]])}), ('max_engine_coolant_temp_attack.log', {9.5e-05: (0.9894626103752759, 0.0, 0.0, 0, [[0, 569], [42, 57373]]), 8.5e-05: (0.9894626103752759, 0.0, 0.0, 0, [[0, 569], [42, 57373]]), 7.5e-05: (0.9894626103752759, 0.0, 0.0, 0, [[0, 569], [42, 57373]]), 6.5e-05: (0.9894108719646799, 0.0, 0.0, 0, [[0, 572], [42, 57370]]), 5.5e-05: (0.9894108719646799, 0.0, 0.0, 0, [[0, 572], [42, 57370]])}), ('max_speedometer_attack_1.log', {9.5e-05: (0.9796565099168266, 0.0, 0.0, 0, [[0, 1626], [2444, 195994]]), 8.5e-05: (0.9771473128598849, 0.0, 0.0, 0, [[0, 2128], [2444, 195492]]), 7.5e-05: (0.9769323816378759, 0.0, 0.0, 0, [[0, 2171], [2444, 195449]]), 6.5e-05: (0.9769173864363404, 0.0, 0.0, 0, [[0, 2174], [2444, 195446]]), 5.5e-05: (0.9767674344209852, 0.0, 0.0, 0, [[0, 2204], [2444, 195416]])}), ('max_speedometer_attack_2.log', {9.5e-05: (0.9710740177736202, 0.0, 0.0, 0, [[0, 818], [3140, 132874]]), 8.5e-05: (0.9710740177736202, 0.0, 0.0, 0, [[0, 818], [3140, 132874]]), 7.5e-05: (0.9710740177736202, 0.0, 0.0, 0, [[0, 818], [3140, 132874]]), 6.5e-05: (0.9710301683816651, 0.0, 0.0, 0, [[0, 824], [3140, 132868]]), 5.5e-05: (0.9710301683816651, 0.0, 0.0, 0, [[0, 824], [3140, 132868]])}), ('max_speedometer_attack_3.log', {9.5e-05: (0.9591699265527702, 0.0, 0.0, 0, [[0, 2076], [6107, 192233]]), 8.5e-05: (0.9591449784448347, 0.0, 0.0, 0, [[0, 2081], [6107, 192228]]), 7.5e-05: (0.9591449784448347, 0.0, 0.0, 0, [[0, 2081], [6107, 192228]]), 6.5e-05: (0.9591000718505509, 0.0, 0.0, 0, [[0, 2090], [6107, 192219]]), 5.5e-05: (0.9589503832029379, 0.0, 0.0, 0, [[0, 2120], [6107, 192189]])}), ('reverse_light_off_attack_1.log', {9.5e-05: (0.9784959758551308, 0.0, 0.0, 0, [[0, 696], [672, 62248]]), 8.5e-05: (0.9784959758551308, 0.0, 0.0, 0, [[0, 696], [672, 62248]]), 7.5e-05: (0.9784959758551308, 0.0, 0.0, 0, [[0, 696], [672, 62248]]), 6.5e-05: (0.9784016599597586, 0.0, 0.0, 0, [[0, 702], [672, 62242]]), 5.5e-05: (0.9784016599597586, 0.0, 0.0, 0, [[0, 702], [672, 62242]])}), ('reverse_light_off_attack_2.log', {9.5e-05: (0.9631100171232877, 0.0, 0.0, 0, [[0, 1076], [2371, 89993]]), 8.5e-05: (0.9631100171232877, 0.0, 0.0, 0, [[0, 1076], [2371, 89993]]), 7.5e-05: (0.9631100171232877, 0.0, 0.0, 0, [[0, 1076], [2371, 89993]]), 6.5e-05: (0.9630244006849316, 0.0, 0.0, 0, [[0, 1084], [2371, 89985]]), 5.5e-05: (0.9630244006849316, 0.0, 0.0, 0, [[0, 1084], [2371, 89985]])}), ('reverse_light_off_attack_3.log', {9.5e-05: (0.9707338866004361, 0.0, 0.0, 0, [[0, 1431], [2434, 128199]]), 8.5e-05: (0.9707338866004361, 0.0, 0.0, 0, [[0, 1431], [2434, 128199]]), 7.5e-05: (0.9707338866004361, 0.0, 0.0, 0, [[0, 1431], [2434, 128199]]), 6.5e-05: (0.9707338866004361, 0.0, 0.0, 0, [[0, 1431], [2434, 128199]]), 5.5e-05: (0.9706960261691301, 0.0, 0.0, 0, [[0, 1436], [2434, 128194]])}), ('reverse_light_on_attack_1.log', {9.5e-05: (0.9769328080963362, 0.0, 0.0, 0, [[0, 890], [1991, 122015]]), 8.5e-05: (0.9769248014347938, 0.0, 0.0, 0, [[0, 891], [1991, 122014]]), 7.5e-05: (0.9769248014347938, 0.0, 0.0, 0, [[0, 891], [1991, 122014]]), 6.5e-05: (0.9769248014347938, 0.0, 0.0, 0, [[0, 891], [1991, 122014]]), 5.5e-05: (0.9768687548039969, 0.0, 0.0, 0, [[0, 898], [1991, 122007]])}), ('reverse_light_on_attack_2.log', {9.5e-05: (0.966156437851464, 0.0, 0.0, 0, [[0, 1896], [3689, 159439]]), 8.5e-05: (0.966156437851464, 0.0, 0.0, 0, [[0, 1896], [3689, 159439]]), 7.5e-05: (0.966156437851464, 0.0, 0.0, 0, [[0, 1896], [3689, 159439]]), 6.5e-05: (0.966156437851464, 0.0, 0.0, 0, [[0, 1896], [3689, 159439]]), 5.5e-05: (0.9660716017064185, 0.0, 0.0, 0, [[0, 1910], [3689, 159425]])}), ('reverse_light_on_attack_3.log', {9.5e-05: (0.972242727471566, 0.0, 0.0, 0, [[0, 1710], [2351, 142243]]), 8.5e-05: (0.972242727471566, 0.0, 0.0, 0, [[0, 1710], [2351, 142243]]), 7.5e-05: (0.9722358923884514, 0.0, 0.0, 0, [[0, 1711], [2351, 142242]]), 6.5e-05: (0.9722222222222222, 0.0, 0.0, 0, [[0, 1713], [2351, 142240]]), 5.5e-05: (0.9721880468066492, 0.0, 0.0, 0, [[0, 1718], [2351, 142235]])})]

# get thresholds list 
thresholds = list(list_of_results[0][1].keys())

for filename, results in list_of_results:
    print(f"Filename: {filename}")
    for threshold, result in results.items():
        accuracy, precision, recall, f1_score, confusion_matrix = result
print(thresholds)
        



[9.5e-05, 8.5e-05, 7.5e-05, 6.5e-05, 5.5e-05]


In [ ]:

# [('correlated_signal_attack_1.log',
#   {9.5e-05: (0.9597595548089038, 0.0, 0.0, 0, [[0, 980], [2086, 73126]]),
#    8.5e-05: (0.9597595548089038, 0.0, 0.0, 0, [[0, 980], [2086, 73126]]),
#    7.5e-05: (0.9597595548089038, 0.0, 0.0, 0, [[0, 980], [2086, 73126]]),
#    6.5e-05: (0.9597595548089038, 0.0, 0.0, 0, [[0, 980], [2086, 73126]]),
#    5.5e-05: (0.9597595548089038, 0.0, 0.0, 0, [[0, 980], [2086, 73126]])}),
#  ('correlated_signal_attack_2.log',
#   {9.5e-05: (0.9562928501469148, 0.0, 0.0, 0, [[0, 717], [2139, 62488]]),
#    8.5e-05: (0.9562928501469148, 0.0, 0.0, 0, [[0, 717], [2139, 62488]]),
#    7.5e-05: (0.9562928501469148, 0.0, 0.0, 0, [[0, 717], [2139, 62488]]),
#    6.5e-05: (0.9562622428991185, 0.0, 0.0, 0, [[0, 719], [2139, 62486]]),
#    5.5e-05: (0.9562469392752204, 0.0, 0.0, 0, [[0, 720], [2139, 62485]])}),
#  ('correlated_signal_attack_3.log',
#   {9.5e-05: (0.9578316326530613, 0.0, 0.0, 0, [[0, 390], [1263, 37547]]),
#    8.5e-05: (0.9578061224489796, 0.0, 0.0, 0, [[0, 391], [1263, 37546]]),
#    7.5e-05: (0.9578061224489796, 0.0, 0.0, 0, [[0, 391], [1263, 37546]]),
#    6.5e-05: (0.9578061224489796, 0.0, 0.0, 0, [[0, 391], [1263, 37546]]),
#    5.5e-05: (0.9578061224489796, 0.0, 0.0, 0, [[0, 391], [1263, 37546]])}),
#  ('fuzzing_attack_1.log',
#   {9.5e-05: (0.9910745614035088,
#     0.002680965147453083,
#     0.027777777777777776,
#     0.004889975550122249,
#     [[1, 372], [35, 45192]]),
#    8.5e-05: (0.9910745614035088,
#     0.002680965147453083,
#     0.027777777777777776,
#     0.004889975550122249,
#     [[1, 372], [35, 45192]]),
#    7.5e-05: (0.9910745614035088,
#     0.002680965147453083,
#     0.027777777777777776,
#     0.004889975550122249,
#     [[1, 372], [35, 45192]]),
#    6.5e-05: (0.9910745614035088,
#     0.002680965147453083,
#     0.027777777777777776,
#     0.004889975550122249,
#     [[1, 372], [35, 45192]]),
#    5.5e-05: (0.9910745614035088,
#     0.002680965147453083,
#     0.027777777777777776,
#     0.004889975550122249,
#     [[1, 372], [35, 45192]])}),
#  ('fuzzing_attack_2.log',
#   {9.5e-05: (0.97625,
#     0.004322766570605188,
#     0.2,
#     0.00846262341325811,
#     [[3, 691], [12, 28894]]),
#    8.5e-05: (0.97625,
#     0.004322766570605188,
#     0.2,
#     0.00846262341325811,
#     [[3, 691], [12, 28894]]),
#    7.5e-05: (0.97625,
#     0.004322766570605188,
#     0.2,
#     0.00846262341325811,
#     [[3, 691], [12, 28894]]),
#    6.5e-05: (0.9761824324324324,
#     0.004310344827586207,
#     0.2,
#     0.008438818565400843,
#     [[3, 693], [12, 28892]]),
#    5.5e-05: (0.9761824324324324,
#     0.004310344827586207,
#     0.2,
#     0.008438818565400843,
#     [[3, 693], [12, 28892]])}),
#  ('fuzzing_attack_3.log',
#   {9.5e-05: (0.9901666666666666,
#     0.008547008547008548,
#     0.3333333333333333,
#     0.016666666666666666,
#     [[1, 116], [2, 11881]]),
#    8.5e-05: (0.9901666666666666,
#     0.008547008547008548,
#     0.3333333333333333,
#     0.016666666666666666,
#     [[1, 116], [2, 11881]]),
#    7.5e-05: (0.9901666666666666,
#     0.008547008547008548,
#     0.3333333333333333,
#     0.016666666666666666,
#     [[1, 116], [2, 11881]]),
#    6.5e-05: (0.9901666666666666,
#     0.008547008547008548,
#     0.3333333333333333,
#     0.016666666666666666,
#     [[1, 116], [2, 11881]]),
#    5.5e-05: (0.9901666666666666,
#     0.008547008547008548,
#     0.3333333333333333,
#     0.016666666666666666,
#     [[1, 116], [2, 11881]])}),
#  ('max_engine_coolant_temp_attack.log',
#   {9.5e-05: (0.9894626103752759, 0.0, 0.0, 0, [[0, 569], [42, 57373]]),
#    8.5e-05: (0.9894626103752759, 0.0, 0.0, 0, [[0, 569], [42, 57373]]),
#    7.5e-05: (0.9894626103752759, 0.0, 0.0, 0, [[0, 569], [42, 57373]]),
#    6.5e-05: (0.9894108719646799, 0.0, 0.0, 0, [[0, 572], [42, 57370]]),
#    5.5e-05: (0.9894108719646799, 0.0, 0.0, 0, [[0, 572], [42, 57370]])}),
#  ('max_speedometer_attack_1.log',
#   {9.5e-05: (0.9796565099168266, 0.0, 0.0, 0, [[0, 1626], [2444, 195994]]),
#    8.5e-05: (0.9771473128598849, 0.0, 0.0, 0, [[0, 2128], [2444, 195492]]),
#    7.5e-05: (0.9769323816378759, 0.0, 0.0, 0, [[0, 2171], [2444, 195449]]),
#    6.5e-05: (0.9769173864363404, 0.0, 0.0, 0, [[0, 2174], [2444, 195446]]),
#    5.5e-05: (0.9767674344209852, 0.0, 0.0, 0, [[0, 2204], [2444, 195416]])}),
#  ('max_speedometer_attack_2.log',
#   {9.5e-05: (0.9710740177736202, 0.0, 0.0, 0, [[0, 818], [3140, 132874]]),
#    8.5e-05: (0.9710740177736202, 0.0, 0.0, 0, [[0, 818], [3140, 132874]]),
#    7.5e-05: (0.9710740177736202, 0.0, 0.0, 0, [[0, 818], [3140, 132874]]),
#    6.5e-05: (0.9710301683816651, 0.0, 0.0, 0, [[0, 824], [3140, 132868]]),
#    5.5e-05: (0.9710301683816651, 0.0, 0.0, 0, [[0, 824], [3140, 132868]])}),
#  ('max_speedometer_attack_3.log',
#   {9.5e-05: (0.9591699265527702, 0.0, 0.0, 0, [[0, 2076], [6107, 192233]]),
#    8.5e-05: (0.9591449784448347, 0.0, 0.0, 0, [[0, 2081], [6107, 192228]]),
#    7.5e-05: (0.9591449784448347, 0.0, 0.0, 0, [[0, 2081], [6107, 192228]]),
#    6.5e-05: (0.9591000718505509, 0.0, 0.0, 0, [[0, 2090], [6107, 192219]]),
#    5.5e-05: (0.9589503832029379, 0.0, 0.0, 0, [[0, 2120], [6107, 192189]])}),
#  ('reverse_light_off_attack_1.log',
#   {9.5e-05: (0.9784959758551308, 0.0, 0.0, 0, [[0, 696], [672, 62248]]),
#    8.5e-05: (0.9784959758551308, 0.0, 0.0, 0, [[0, 696], [672, 62248]]),
#    7.5e-05: (0.9784959758551308, 0.0, 0.0, 0, [[0, 696], [672, 62248]]),
#    6.5e-05: (0.9784016599597586, 0.0, 0.0, 0, [[0, 702], [672, 62242]]),
#    5.5e-05: (0.9784016599597586, 0.0, 0.0, 0, [[0, 702], [672, 62242]])}),
#  ('reverse_light_off_attack_2.log',
#   {9.5e-05: (0.9631100171232877, 0.0, 0.0, 0, [[0, 1076], [2371, 89993]]),
#    8.5e-05: (0.9631100171232877, 0.0, 0.0, 0, [[0, 1076], [2371, 89993]]),
#    7.5e-05: (0.9631100171232877, 0.0, 0.0, 0, [[0, 1076], [2371, 89993]]),
#    6.5e-05: (0.9630244006849316, 0.0, 0.0, 0, [[0, 1084], [2371, 89985]]),
#    5.5e-05: (0.9630244006849316, 0.0, 0.0, 0, [[0, 1084], [2371, 89985]])}),
#  ('reverse_light_off_attack_3.log',
#   {9.5e-05: (0.9707338866004361, 0.0, 0.0, 0, [[0, 1431], [2434, 128199]]),
#    8.5e-05: (0.9707338866004361, 0.0, 0.0, 0, [[0, 1431], [2434, 128199]]),
#    7.5e-05: (0.9707338866004361, 0.0, 0.0, 0, [[0, 1431], [2434, 128199]]),
#    6.5e-05: (0.9707338866004361, 0.0, 0.0, 0, [[0, 1431], [2434, 128199]]),
#    5.5e-05: (0.9706960261691301, 0.0, 0.0, 0, [[0, 1436], [2434, 128194]])}),
#  ('reverse_light_on_attack_1.log',
#   {9.5e-05: (0.9769328080963362, 0.0, 0.0, 0, [[0, 890], [1991, 122015]]),
#    8.5e-05: (0.9769248014347938, 0.0, 0.0, 0, [[0, 891], [1991, 122014]]),
#    7.5e-05: (0.9769248014347938, 0.0, 0.0, 0, [[0, 891], [1991, 122014]]),
#    6.5e-05: (0.9769248014347938, 0.0, 0.0, 0, [[0, 891], [1991, 122014]]),
#    5.5e-05: (0.9768687548039969, 0.0, 0.0, 0, [[0, 898], [1991, 122007]])}),
#  ('reverse_light_on_attack_2.log',
#   {9.5e-05: (0.966156437851464, 0.0, 0.0, 0, [[0, 1896], [3689, 159439]]),
#    8.5e-05: (0.966156437851464, 0.0, 0.0, 0, [[0, 1896], [3689, 159439]]),
#    7.5e-05: (0.966156437851464, 0.0, 0.0, 0, [[0, 1896], [3689, 159439]]),
#    6.5e-05: (0.966156437851464, 0.0, 0.0, 0, [[0, 1896], [3689, 159439]]),
#    5.5e-05: (0.9660716017064185, 0.0, 0.0, 0, [[0, 1910], [3689, 159425]])}),
#  ('reverse_light_on_attack_3.log',
#   {9.5e-05: (0.972242727471566, 0.0, 0.0, 0, [[0, 1710], [2351, 142243]]),
#    8.5e-05: (0.972242727471566, 0.0, 0.0, 0, [[0, 1710], [2351, 142243]]),
#    7.5e-05: (0.9722358923884514, 0.0, 0.0, 0, [[0, 1711], [2351, 142242]]),
#    6.5e-05: (0.9722222222222222, 0.0, 0.0, 0, [[0, 1713], [2351, 142240]]),
#    5.5e-05: (0.9721880468066492, 0.0, 0.0, 0, [[0, 1718], [2351, 142235]])})]

In [ ]:
# {
#     .000095: [
#         ('correlated_signal_attack_1.log', (0.9597595548089038, 0.0, 0.0, 0, [[0, 980], [2086, 73126]])),
#         ('correlated_signal_attack_2.log', (0.9562928501469148, 0.0, 0.0, 0, [[0, 717], [2139, 62488]])), 
#         ('correlated_signal_attack_3.log', (0.9578316326530613, 0.0, 0.0, 0, [[0, 390], [1263, 37547]])), 
#         ('fuzzing_attack_1.log', (0.9910745614035088, 0.002680965147453083, 0.027777777777777776, 0.004889975550122249, [[1, 372], [35, 45192]])), 
#         ('fuzzing_attack_2.log', (0.97625, 0.004322766570605188, 0.2, 0.00846262341325811, [[3, 691], [12, 28894]])), 
#         ('fuzzing_attack_3.log', (0.9901666666666666, 0.008547008547008548, 0.3333333333333333, 0.016666666666666666, [[1, 116], [2, 11881]])), 
#         ('max_engine_coolant_temp_attack.log', (0.9894626103752759, 0.0, 0.0, 0, [[0, 569], [42, 57373]])), 
#         ('max_speedometer_attack_1.log', (0.9796565099168266, 0.0, 0.0, 0, [[0, 1626], [2444, 195994]])), 
#         ('max_speedometer_attack_2.log', (0.9710740177736202, 0.0, 0.0, 0, [[0, 818], [3140, 132874]])), 
#         ('max_speedometer_attack_3.log', (0.9591699265527702, 0.0, 0.0, 0, [[0, 2076], [6107, 192233]])), 
#         ('reverse_light_off_attack_1.log', (0.9784959758551308, 0.0, 0.0, 0, [[0, 696], [672, 62248]])),
#         ('reverse_light_off_attack_2.log', (0.9631100171232877, 0.0, 0.0, 0, [[0, 1076], [2371, 89993]])), 
#         ('reverse_light_off_attack_3.log', (0.9707338866004361, 0.0, 0.0, 0, [[0, 1431], [2434, 128199]])), 
#         ('reverse_light_on_attack_1.log', (0.9769328080963362, 0.0, 0.0, 0, [[0, 890], [1991, 122015]])), 
#         ('reverse_light_on_attack_2.log', (0.966156437851464, 0.0, 0.0, 0, [[0, 1896], [3689, 159439]])), 
#         ('reverse_light_on_attack_3.log', (0.972242727471566, 0.0, 0.0, 0, [[0, 1710], [2351, 142243]]))
#         ],
#     .000085: data..
# }

# # structure:
# (filename, (accuracy, precision, recall, f1_score, confusion_matrix)
# ex:
# ('reverse_light_on_attack_3.log', (0.972242727471566, 0.0, 0.0, 0, [[0, 1710], [2351, 142243]]))

# graph thresholds vs avg f1 score for that group


# for the best performing threshold make a graph for each filename and show the results




In [ ]:
detector.threshold = 5.2644566676462956e-05


In [4]:
from tqdm import tqdm

meta_results = {}
for threshold in [.000095, .000085, .000075, .000065, .000055]:
    results = {}
    for i, loader in enumerate(attack_loaders):
        filename = loader.can_data[0].filename[0]
        print(f"Testing on {filename}")
        results[filename] = calculate_metrics(detector.detect_attacks(loader))
        print(results[filename])
    meta_results[threshold] = results

print(meta_results)

NameError: name 'attack_loaders' is not defined